In [1]:
import sys
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import glob
import scipy.ndimage

In [2]:
sys.path.append('../Preprocess')
import CTPPreprocess
import CalcParaMaps

In [3]:
filelist = np.load('../../filelist_uncor.npy')
imgDir = '/home/dwu/data/isles/TRAINING/'
outDir = '/home/dwu/trainData/Noise2Noise/train/ctp/real/data/'

In [4]:
if not os.path.exists(outDir):
    os.makedirs(outDir)

In [5]:
# ct perfusion images
imgs4d = []
for case in filelist:
    name = glob.glob(os.path.join(imgDir, 'case_'+case, '*', '*4DPWI*.nii'))[0]
    ctpImg = nib.load(name).get_fdata().astype(np.float32).transpose(2, 1, 0, 3)
    imgs4d.append(ctpImg)

# np.save(os.path.join(outDir, 'imgs4d'), imgs4d)

In [6]:
def RetrieveParamaps(srcPattern, dstName, imgDir, outDir, filelist):
    imgs = []
    for case in filelist:
        name = glob.glob(os.path.join(imgDir, 'case_'+case, '*', srcPattern))[0]
        img = nib.load(name).get_fdata().astype(np.float32).transpose(2, 1, 0)[..., np.newaxis]
        imgs.append(img)

    np.save(os.path.join(outDir, dstName), imgs)
    
    return imgs

In [7]:
nccts = RetrieveParamaps('*O.CT*.nii', 'ncct', imgDir, outDir, filelist)
cbf0 = RetrieveParamaps('*CBF*.nii', 'cbf0', imgDir, outDir, filelist)
cbv0 = RetrieveParamaps('*CBV*.nii', 'cbv0', imgDir, outDir, filelist)
mtt0 = RetrieveParamaps('*MTT*.nii', 'mtt0', imgDir, outDir, filelist)
tmax0 = RetrieveParamaps('*Tmax*.nii', 'tmax0', imgDir, outDir, filelist)
ots = RetrieveParamaps('*O.OT*.nii', 'ot', imgDir, outDir, filelist)

In [8]:
# get bone mask
masks = []
for i in range(len(nccts)):
    mask = CTPPreprocess.BrainMask(nccts[i][...,0].transpose(2, 1, 0)).transpose(2,1,0)[...,np.newaxis]
    masks.append(mask)
    
# np.save(os.path.join(outDir, 'mask'), masks)

In [9]:
# get aif and vof
aifs = []
vofs = []
for i in range(len(imgs4d)):
    tac = (imgs4d[i] - nccts[i]) * masks[i]
    tac = tac.transpose(2,1,0,3)
    mask = masks[i].transpose(2,1,0,3)[...,0]
    
    vof, aif, _, _, _ = CTPPreprocess.AutoVofAndAif(tac, mask)
    
    aifs.append(aif)
    vofs.append(vof)

# np.save(os.path.join(outDir, 'vof'), vofs)
# np.save(os.path.join(outDir, 'aif'), aifs)

In [10]:
# Calculate cbf and get cbf calibration factors
cbfFacs = []
cbvFacs = []
for i in range(len(imgs4d)):
    tac = (imgs4d[i] - nccts[i]) * masks[i]
    tac = scipy.ndimage.filters.gaussian_filter1d(tac, 1.75, 1)
    tac = scipy.ndimage.filters.gaussian_filter1d(tac, 1.75, 2)
    
    cbf, cbv, mtt = CalcParaMaps.CalcParaMaps(tac, masks[i], vof = vofs[i], aif = aifs[i])    
    
    cbfFac = np.sum(cbf0[i] * masks[i]) / np.sum(cbf[...,np.newaxis] * masks[i])
    cbfFacs.append(cbfFac)
    
    cbvFac = np.sum(cbv0[i] * masks[i]) / np.sum(cbv[...,np.newaxis] * masks[i])
    cbvFacs.append(cbvFac)

# np.save(os.path.join(outDir, 'cbfFac'), cbfFacs)
np.save(os.path.join(outDir, 'cbvFac'), cbvFacs)